In [1]:
import csv
import json
from abc import ABC, abstractmethod

# --- ส่วนที่ 1: การออกแบบ Class (OOP) ---

class Product(ABC): # ใช้ ABC เพื่อทำเป็น Abstract Base Class (ตามหลักการ OOP)
    def __init__(self, p_id, name, price, stock):
        self._id = p_id          # Protected Attribute
        self._name = name        # Protected Attribute
        self._price = price      # Protected Attribute
        self._stock = int(stock) # Protected Attribute

    # Encapsulation: Getter/Setter สำหรับ price
    @property
    def price(self):
        return self._price

    @price.setter
    def price(self, value):
        if value < 0:
            raise ValueError("Price cannot be negative")
        self._price = value

    @property
    def name(self):
        return self._name
    
    @property
    def stock(self):
        return self._stock

    def update_stock(self, amount):
        """เพิ่มหรือลดสต็อก (ส่งค่าลบเพื่อลด)"""
        new_stock = self._stock + amount
        if new_stock < 0:
            print(f"Error: Not enough stock for {self._name}")
            return False
        self._stock = new_stock
        return True

    @abstractmethod
    def get_info(self):
        """Method ที่ลูกต้องเขียนทับ (Override)"""
        pass

class PerishableProduct(Product):
    def __init__(self, p_id, name, price, stock, expiry_date):
        super().__init__(p_id, name, price, stock) # เรียกใช้ Constructor แม่
        self.expiry_date = expiry_date

    # Method Overriding
    def get_info(self):
        return f"[Perishable] ID: {self._id} | {self._name} | Stock: {self._stock} | Exp: {self.expiry_date}"

class ElectronicProduct(Product):
    def __init__(self, p_id, name, price, stock, warranty_period):
        super().__init__(p_id, name, price, stock)
        self.warranty_period = warranty_period

    # Method Overriding
    def get_info(self):
        return f"[Electronic] ID: {self._id} | {self._name} | Stock: {self._stock} | Warranty: {self.warranty_period}"


# --- ส่วนที่ 2: การจัดการข้อมูลและไฟล์ ---

def load_inventory(filename):
    products = []
    try:
        with open(filename, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                # Factory Pattern แบบง่ายๆ ในการสร้าง Object
                if row['Type'] == 'Perishable':
                    p = PerishableProduct(
                        p_id=row['ID'],
                        name=row['Name'],
                        price=float(row['Price']),
                        stock=int(row['Stock']),
                        expiry_date=row['Extra_Info'] # Map ข้อมูลตามเงื่อนไข
                    )
                elif row['Type'] == 'Electronic':
                    p = ElectronicProduct(
                        p_id=row['ID'],
                        name=row['Name'],
                        price=float(row['Price']),
                        stock=int(row['Stock']),
                        warranty_period=row['Extra_Info'] # Map ข้อมูลตามเงื่อนไข
                    )
                else:
                    continue # ข้ามประเภทที่ไม่รู้จัก
                
                products.append(p)
    except FileNotFoundError:
        print(f"Error: File {filename} not found.")
    except Exception as e:
        print(f"Error reading file: {e}")
    
    return products

def generate_report(products):
    # Data Processing: คำนวณมูลค่ารวม
    total_value = sum([p.price * p.stock for p in products])
    
    # Data Processing: หาสินค้า Low Stock (สมมติ < 10)
    # ใช้ List Comprehension
    low_stock_items = [p.name for p in products if p.stock < 10]

    # Data Processing: หาข้อความแจ้งเตือน (ตัวอย่างแบบง่าย เช็คปี)
    alert_messages = []
    for p in products:
        if isinstance(p, PerishableProduct):
            # ตัวอย่าง Logic ง่ายๆ: ถ้าปี 2023 ถือว่าหมดอายุแล้ว (ปรับตามวันที่ปัจจุบันจริงได้)
            if "2023" in p.expiry_date: 
                alert_messages.append(f"{p.name} is expired or expiring soon ({p.expiry_date})")

    report_data = {
        "total_value": total_value,
        "product_count": len(products),
        "low_stock_items": low_stock_items,
        "alert_messages": alert_messages
    }

    # บันทึกลง JSON
    try:
        with open('warehouse_report.json', 'w', encoding='utf-8') as f:
            json.dump(report_data, f, indent=4, ensure_ascii=False)
        print("Report generated: warehouse_report.json")
    except Exception as e:
        print(f"Error saving report: {e}")

# --- ส่วน Main Execution ---
if __name__ == "__main__":
    # 1. อ่านไฟล์
    inventory = load_inventory('inventory.csv')
    
    # 2. แสดงผล Console (Polymorphism: get_info ทำงานต่างกันตามชนิด)
    print(f"--- Loaded {len(inventory)} Items ---")
    for item in inventory[:5]: # แสดงแค่ 5 ตัวอย่าง
        print(item.get_info())
    print("...")

    # 3. สร้างรายงาน
    generate_report(inventory)

--- Loaded 200 Items ---
[Perishable] ID: 1001 | Greek Yogurt (200g) | Stock: 97 | Exp: 2026-02-21
[Electronic] ID: 1002 | Mechanical Keyboard Model-493 | Stock: 145 | Warranty: 1 Year
[Electronic] ID: 1003 | External Hard Drive Model-351 | Stock: 49 | Warranty: 6 Months
[Perishable] ID: 1004 | Kimchi (2000g) | Stock: 69 | Exp: 2026-01-31
[Electronic] ID: 1005 | External Hard Drive Model-527 | Stock: 7 | Warranty: 6 Months
...
Report generated: warehouse_report.json
